In [60]:
import random
import math
class Node:
    def __init__(self, weight):
        self.weight = weight
        self.bias = random.random() * 10
        self.threshold = 1
        
    def giveWeight(self):
        return self.weight 
    
    def giveBias(self):
        return self.bias
    
    def calculate(self, values):
        val = 0
        activation = 0
        if self.threshold > values[0]:
            for i in range(len(values)):
                numToAdd = values[i]
                val += (numToAdd * self.weight)

            val += self.bias

            if val > 33:
                val = 32
            elif val < -33:
                val = -32
        else:
            for i in range(len(values)):
                numToAdd = values[i]
                val += (numToAdd - self.weight)

            val += self.bias

            if val > 33:
                val = 32
            elif val < -33:
                val = -32
                
        activation = 1/(pow(2.718, val))
        
        if activation <= 0.5:
            return activation
        return -activation
    
    def change(self, direction, weight):
        if (direction == "pos"):
            self.weight += weight
        else:
            self.weight -= weight
    
    def changeB(self, direction, bias):
        if (direction == "pos"):
            self.bias += bias
        else:
            self.bias -= bias
    
    def changeThresh(self, changeby1):
        self.bias += changeby1

    def setWeight(self, newWeight):
        self.weight = newWeight
    
    def setBias(self, newBias):
        self.bias = newBias
    
class Layer:
    layer = []
    def __init__(self, numNodes):
        self.layer = []
        self.numNodes = numNodes
        for i in range(numNodes):
            self.layer.append(Node(random.random()))
    
    def forwardProp(self, values):
        ret = []
        for i in range(self.numNodes):
            ret.append(self.layer[i].calculate(values))
        return ret
    
    def inputLayer(self, values):
        ret = []
        for i in range(len(values)):
            temp = []
            temp.append(values[i])
            ret.append(self.layer[i].calculate(temp))
        return ret
    
    def weightsChange(self, direction):
        for i in range(len(self.layer)):
            if direction == "pos":
                self.layer[i].change(0.01, 0)
            else:
                self.layer[i].change(-0.01, 0)
    
class Network:
    network = []
    def __init__(self, depth, inputs, outputs):
        self.network = []
        self.depth = depth
        self.inputs = inputs
        self.outputs = outputs
        self.iterationCap = 100
        self.changeBy = 0.5
        flattenAmt = int(inputs/2)
                
        self.network.append(Layer(inputs))
        
        self.network.append(Layer(flattenAmt))
        
        for i in range(depth):
            count = 0
            if i < int(depth/2):
                self.network.append(Layer(flattenAmt + i))
            else:
                if (count < outputs + 1):
                    self.network.append(Layer(outputs - count))
                    count += 1
            
        self.network.append(Layer(outputs))
        
    def setIterationCap(self, newIterationCap):
        self.iterationCap = newIterationCap
        
    def setChangeBy(self, newChangeBy):
        self.changeBy = newChangeBy
    
    def setWeights(self, weightlist):
        for a in range(len(self.network)):
            for b in range(len(self.network[a].layer)):
                self.network[a].layer[b].setWeight(weightlist[a][b])
                
    def setBiases(self, biaslist):
        for a in range(len(self.network)):
            for b in range(len(self.network[a].layer)):
                self.network[a].layer[b].setBias(weightlist[a][b])
                
    def getWeights(self):
        temp = []
        for a in range(len(self.network)):
            microarray = []
            for b in range(len(self.network[a].layer)):
                microarray.append(self.network[a].layer[b].giveWeight())
            temp.append(microarray)
        return temp
    
    def getBiases(self):
        temp = []
        for a in range(len(self.network)):
            microarray = []
            for b in range(len(self.network[a].layer)):
                microarray.append(self.network[a].layer[b].giveBias())
            temp.append(microarray)
        return temp
                
    def printWeights(self):
        for i in range(len(self.network)):
            print("-----")
            for j in range(len(self.network[i].layer)):
                print("weight:", self.network[i].layer[j].weight)
                print()
                
    def printBiases(self):
        for i in range(len(self.network)):
            print("------")
            for j in range(len(self.network[i].layer)):
                print("bias:", self.network[i].layer[j].bias)
                print()
                
    def forwardPropagate(self, userInput):
        #convert user input
        converted = []
        for i in range(len(userInput)):
            for j in range(len(userInput[i])):
                converted.append(userInput[i][j] * (i + j + 1))
        ret = self.network[0].inputLayer(converted)
        for i in range(1, self.depth):
            ret = self.network[i].forwardProp(ret)
        return ret

    def calculateError1(self, userInputs, desiredResults):
        error = 0
        for i in range(len(userInputs)):
            temp = self.forwardPropagate(userInputs[i])
            if desiredResults[i] != temp:
                error += (abs(desiredResults[i][0] - temp[0]) + abs(desiredResults[i][1] - temp[1]))
        return error
    
    def calculateError(self, userInputs, desiredResults):
        ierror = 0
        jerror = 0
        for i in range(len(userInputs)):
            temp = self.forwardPropagate(userInputs[i])
            if desiredResults[i] != temp:
                ierror += (abs(desiredResults[i][1] - temp[1]))
            else:
                jerror += (abs(desiredResults[i][0] - temp[0]))
                
        ierror /= (len(desiredResults) * len(desiredResults[1]))
        jerror /= (len(desiredResults) * len(desiredResults[0]))
        
        return ((ierror * (1 - ierror)) + (jerror * (1 - jerror)))
    
    def backPropagate(self, userInputs, desiredResults, percentAccurate):    
        iterationCap = self.iterationCap
        changeBy = self.changeBy
        
        layerOn = 0
        error = self.calculateError(userInputs, desiredResults)
        initialError = error
        while (layerOn < len(self.network)):
            nodeOn = 0

            while (nodeOn < len(self.network[layerOn].layer)):
                #forward until not better
                improved = True
                count = 0
                countI = 0
                for j in range(iterationCap):
                    self.network[layerOn].layer[nodeOn].change("pos", changeBy)
                    newerror = self.calculateError(userInputs, desiredResults)
                    count += 1

                    if (newerror*newerror) < (error * error):
                        error = newerror
                        improved = True
                        countI = count
                    else:
                        improved = False
                if improved == False:
                    j = iterationCap
                    while (j > countI):
                        self.network[layerOn].layer[nodeOn].change("neg", changeBy)
                        j -= 1

                #backward until not better
                improved = True
                count = 0
                countI = 0
                for j in range(iterationCap):
                    self.network[layerOn].layer[nodeOn].change("neg", changeBy)
                    newerror = self.calculateError(userInputs, desiredResults)
                    count += 1

                    if (newerror*newerror) < (error * error):
                        error = newerror
                        improved = True
                        countI = count
                    else:
                        improved = False
                if improved == False:
                    j = iterationCap
                    while (j > countI):
                        self.network[layerOn].layer[nodeOn].change("pos", changeBy)
                        j -= 1

                nodeOn += 1
            layerOn += 1

        ########
        iterationCap = self.iterationCap
        changeBy = self.changeBy

        layerOn = 0
        error = self.calculateError(userInputs, desiredResults)
        initialError = error
        while (layerOn < len(self.network)):
            nodeOn = 0

            while (nodeOn < len(self.network[layerOn].layer)):
                #forward until not better
                improved = True
                count = 0
                countI = 0
                for j in range(iterationCap):
                    self.network[layerOn].layer[nodeOn].changeB("pos", changeBy)
                    newerror = self.calculateError(userInputs, desiredResults)
                    count += 1

                    if (newerror*newerror) < (error * error):
                        error = newerror
                        improved = True
                        countI = count
                    else:
                        improved = False
                if improved == False:
                    j = iterationCap
                    while (j > countI):
                        self.network[layerOn].layer[nodeOn].changeB("neg", changeBy)
                        j -= 1

                #backward until not better
                improved = True
                count = 0
                countI = 0
                for j in range(iterationCap):
                    self.network[layerOn].layer[nodeOn].changeB("neg", changeBy)
                    newerror = self.calculateError(userInputs, desiredResults)
                    count += 1
                    
                    if (newerror*newerror) < (error * error):
                        error = newerror
                        improved = True
                        countI = count
                    else:
                        improved = False
                if improved == False:
                    j = iterationCap
                    while (j > countI):
                        self.network[layerOn].layer[nodeOn].changeB("pos", changeBy)
                        j -= 1

                nodeOn += 1
            layerOn += 1
        ########
        iterationCap = self.iterationCap
        changeBy = self.changeBy

        layerOn = 0
        error = self.calculateError(userInputs, desiredResults)
        initialError = error
        while (layerOn < len(self.network)):
            nodeOn = 0

            while (nodeOn < len(self.network[layerOn].layer)):
                #forward until not better
                improved = True
                count = 0
                countI = 0
                for j in range(iterationCap):
                    self.network[layerOn].layer[nodeOn].changeThresh(0.5)
                    newerror = self.calculateError(userInputs, desiredResults)
                    count += 1

                    if (newerror*newerror) < (error * error):
                        error = newerror
                        improved = True
                        countI = count
                    else:
                        improved = False
                if improved == False:
                    j = iterationCap
                    while (j > countI):
                        self.network[layerOn].layer[nodeOn].changeThresh(-0.5)
                        j -= 1

                #backward until not better
                improved = True
                count = 0
                countI = 0
                for j in range(iterationCap):
                    self.network[layerOn].layer[nodeOn].changeThresh(-0.5)
                    newerror = self.calculateError(userInputs, desiredResults)
                    count += 1
                    
                    if (newerror*newerror) < (error * error):
                        error = newerror
                        improved = True
                        countI = count
                    else:
                        improved = False
                if improved == False:
                    j = iterationCap
                    while (j > countI):
                        self.network[layerOn].layer[nodeOn].changeThresh(0.5)
                        j -= 1

                nodeOn += 1
            layerOn += 1
        
                
    def userTest(self, userInput):
        ret = self.forwardPropagate(userInput)
        print(ret[0], ret[1])

        if (ret[0] < ret[1]):
            print("this is a square")
        elif (ret[0] > ret[1]):
            print("this is not a square")
        else:
            print("this was not supposed to happen")
            
listInputs = []
answerInputs = []

answerInputs.append([0, 1])
arr1 = [[1, 1, 0],
        [1, 1, 0],
        [0, 0, 0]]
listInputs.append(arr1)

answerInputs.append([0, 1])
arr2 = [[0, 0, 0],
        [0, 1, 1],
        [0, 1, 1]]
listInputs.append(arr2)

answerInputs.append([1, 0])
arr3 = [[1, 1, 0],
        [1, 1, 0],
        [1, 1, 0]]
listInputs.append(arr3)

answerInputs.append([1, 0])
arr4 = [[0, 0, 1],
        [1, 1, 1],
        [0, 1, 0]]
listInputs.append(arr4)

answerInputs.append([1, 0])
arr5 = [[1, 0, 0],
        [1, 1, 0],
        [0, 0, 1]]
listInputs.append(arr5)

answerInputs.append([1, 0])
arr6 = [[1, 1, 1],
        [1, 0, 0],
        [0, 1, 0]]
listInputs.append(arr6)

answerInputs.append([1, 0])
arr7 = [[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]]
listInputs.append(arr7)

answerInputs.append([0, 1])
arr8 = [[0, 0, 0],
        [1, 1, 0],
        [1, 1, 0]]
listInputs.append(arr8)

answerInputs.append([0, 1])
arr9 = [[1, 1, 0],
        [1, 1, 0],
        [0, 0, 0]]
listInputs.append(arr9)

answerInputs.append([0, 1])
arr10 = [[0, 0, 0],
         [0, 1, 1],
         [0, 1, 1]]
listInputs.append(arr10)

##############################
trainset1 = [[1, 0, 0],
             [1, 1, 1],
             [1, 1, 0]]
trainset2 = [[1, 0, 0],
             [1, 0, 1],
             [0, 0, 1]]
trainset3 = [[0, 0, 0],
             [0, 1, 1],
             [0, 1, 1]]
trainset4 = [[0, 1, 1],
             [0, 1, 1],
             [0, 0, 0]]
trainset5 = [[0, 0, 0],
             [0, 0, 0],
             [0, 0, 0]]

neuralNet = Network(6, 9, 2)

print("this is error before: ", neuralNet.calculateError(listInputs, answerInputs))

neuralNet.setIterationCap(100)
neuralNet.setChangeBy(.61803398875/1.61803398875) #best of .25 error
#neuralNet.setChangeBy(2.718/11)
neuralNet.backPropagate(listInputs, answerInputs, 0.9)
print("this is error after: ", neuralNet.calculateError(listInputs, answerInputs))

neuralNet.userTest(trainset1)
neuralNet.userTest(trainset2)
neuralNet.userTest(trainset3)
neuralNet.userTest(trainset4)

neuralNet.setWeights(neuralNet.getWeights())

this is error before:  0.1875000409729483
this is error after:  -4.508780018043315e-10
0.08432815063527975 -0.8738506325130628
this is not a square
0.042740230907543246 0.4935919847216829
this is a square
0.017348285575885046 0.23133154103217127
this is a square
0.017351978775760266 0.23137293381717872
this is a square


0
